In [30]:
import pandas as pd
import numpy as np
pd.set_option('max_rows', 10)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
print("Vis setup Complete")
print("Setup complete.")

Vis setup Complete
Setup complete.


In [31]:
loan_data_path = "J:\LendingClubLoans2018-2020.csv";
loan_data = pd.read_csv(loan_data_path)
loan_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77159 entries, 0 to 77158
Data columns (total 108 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   id                                          int64  
 1   emp_title                                   object 
 2   application_type                            object 
 3   home_ownership                              object 
 4   loan_status                                 object 
 5   loan_amnt                                   int64  
 6   int_rate                                    object 
 7   term                                        object 
 8   installment                                 float64
 9   grade                                       object 
 10  earliest_cr_line                            object 
 11  issue_d                                     object 
 12  last_pymnt_d                                object 
 13  next_pymnt_d                  

In [32]:
feature_missing_values = loan_data.isna().sum()
feature_missing_values = feature_missing_values.to_frame('Missing_Values')

In [33]:
percenatges_df = (loan_data.isna().sum()* 100 / len(loan_data)).to_frame('Missing_Percentages')
missing_values_stats_df = pd.concat([feature_missing_values, percenatges_df], axis=1)
missing_values_stats_df


,Missing_Values,Missing_Percentages
id,0,0.000000
emp_title,5862,7.597299
application_type,0,0.000000
home_ownership,0,0.000000
loan_status,0,0.000000
...,...,...
deferral_term,72668,94.179551
hardship_amount,71042,92.072215
hardship_payoff_balance_amount,71042,92.072215
hardship_last_payment_amount,71042,92.072215


In [34]:
missing_values_stats_df.loc['deferral_term']

Missing_Values         72668.000000
Missing_Percentages       94.179551
Name: deferral_term, dtype: float64

In [35]:
missing_values_stats_df[missing_values_stats_df['Missing_Percentages'] > 60]

,Missing_Values,Missing_Percentages
annual_inc_joint,67777,87.840693
verification_status_joint,67780,87.844581
hardship_reason,72668,94.179551
hardship_type,72668,94.179551
hardship_status,72668,94.179551
...,...,...
deferral_term,72668,94.179551
hardship_amount,71042,92.072215
hardship_payoff_balance_amount,71042,92.072215
hardship_last_payment_amount,71042,92.072215


In [36]:
more_null_values = []
for index, row in missing_values_stats_df.iterrows():
    if (row['Missing_Percentages'] > 60):
        more_null_values.append(index)

In [37]:
print('Number of Features which has more than 60 peercent of null values: ', len(more_null_values))
more_null_values

Number of Features which has more than 60 peercent of null values:  17


['annual_inc_joint',
 'verification_status_joint',
 'hardship_reason',
 'hardship_type',
 'hardship_status',
 'payment_plan_start_date',
 'dti_joint',
 'mths_since_last_record',
 'mths_since_last_major_derog',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'revol_bal_joint',
 'deferral_term',
 'hardship_amount',
 'hardship_payoff_balance_amount',
 'hardship_last_payment_amount',
 'orig_projected_additional_accrued_interest']

In [38]:
new_loan_df = loan_data.drop(more_null_values, axis=1)
new_loan_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77159 entries, 0 to 77158
Data columns (total 91 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          77159 non-null  int64  
 1   emp_title                   71297 non-null  object 
 2   application_type            77159 non-null  object 
 3   home_ownership              77159 non-null  object 
 4   loan_status                 77159 non-null  object 
 5   loan_amnt                   77159 non-null  int64  
 6   int_rate                    77159 non-null  object 
 7   term                        77159 non-null  object 
 8   installment                 77159 non-null  float64
 9   grade                       77159 non-null  object 
 10  earliest_cr_line            77159 non-null  object 
 11  issue_d                     77159 non-null  object 
 12  last_pymnt_d                77159 non-null  object 
 13  next_pymnt_d                332

In [39]:
categorical_features = new_loan_df.select_dtypes(exclude=[np.number])
print(len(categorical_features.columns))
categorical_features
# new_loan_df[dtype="string"]

17


,emp_title,application_type,home_ownership,loan_status,int_rate,term,grade,earliest_cr_line,issue_d,last_pymnt_d,next_pymnt_d,verification_status,pymnt_plan,purpose,hardship_flag,initial_list_status,revol_util
0,Stagehand,Individual,RENT,Fully Paid,15.05%,36 months,C,Jan-01,8/1/2017,4/1/2020,NaN,Verified,n,debt_consolidation,N,w,56.30%
1,Detention Deputy,Individual,RENT,Charged Off,14.08%,60 months,C,Dec-00,7/1/2017,6/1/2019,NaN,Not Verified,n,debt_consolidation,N,w,43.50%
2,NaN,Individual,MORTGAGE,Current,14.08%,60 months,C,Apr-86,7/1/2017,5/1/2020,Jun-20,Verified,n,home_improvement,N,w,9.70%
3,Driver,Individual,RENT,Fully Paid,19.03%,36 months,D,Feb-90,8/1/2017,10/1/2019,NaN,Source Verified,n,debt_consolidation,N,w,86.90%
4,JP Morgan Chase,Individual,MORTGAGE,Fully Paid,8.90%,60 months,A,Mar-88,8/1/2013,1/1/2019,NaN,Verified,n,debt_consolidation,N,w,24.20%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77154,IT analyst,Individual,MORTGAGE,Current,15.05%,36 months,C,Mar-95,9/1/2017,5/1/2020,Jun-20,Verified,n,other,N,w,77%
77155,Principal IA Engineer,Individual,MORTGAGE,Current,7.97%,36 months,A,Dec-02,9/1/2017,5/1/2020,Jun-20,Not Verified,n,home_improvement,N,w,59%
77156,Attorney,Individual,RENT,Fully Paid,5.32%,36 months,A,Aug-99,12/1/2017,5/1/2018,NaN,Not Verified,n,debt_consolidation,N,w,4%
77157,Pastor,Individual,OWN,Fully Paid,20.00%,36 months,D,Jan-07,8/1/2017,6/1/2018,NaN,Source Verified,n,debt_consolidation,N,w,72.70%


In [40]:
categorical_features.describe()

,emp_title,application_type,home_ownership,loan_status,int_rate,term,grade,earliest_cr_line,issue_d,last_pymnt_d,next_pymnt_d,verification_status,pymnt_plan,purpose,hardship_flag,initial_list_status,revol_util
count,71297,77159,77159,77159,77159,77159,77159,77159,77159,77159,33256,77159,77159,77159,75532,77159,77056
unique,28185,2,5,7,129,2,7,622,16,28,9,3,1,13,2,2,1074
top,Teacher,Individual,MORTGAGE,Fully Paid,16.02%,36 months,C,Sep-05,8/1/2017,5/1/2020,Jun-20,Source Verified,n,debt_consolidation,N,w,0%
freq,1658,67777,37334,34289,5181,52800,25152,665,17934,32333,29658,28886,77159,42980,71962,61477,509


In [41]:
categorical_features_of_interest = [
    'emp_title', 
    'application_type', 
    'home_ownership',
    'term',
    'grade',
    'verification_status',
    'purpose',
    'hardship_flag',
    'revol_util',
    'int_rate'
]

import category_encoders as ce
enc = ce.OrdinalEncoder(cols=categorical_features_of_interest, handle_missing='return_nan', return_df= True)

#We now fit the model and transform the data and put it in X which is a dataframe
transformed_loan_df=enc.fit_transform(new_loan_df)
type(transformed_loan_df)

C:\Users\Waqar Ayub\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


pandas.core.frame.DataFrame

In [42]:
transformed_loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77159 entries, 0 to 77158
Data columns (total 91 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          77159 non-null  int64  
 1   emp_title                   71297 non-null  float64
 2   application_type            77159 non-null  float64
 3   home_ownership              77159 non-null  float64
 4   loan_status                 77159 non-null  object 
 5   loan_amnt                   77159 non-null  int64  
 6   int_rate                    77159 non-null  float64
 7   term                        77159 non-null  float64
 8   installment                 77159 non-null  float64
 9   grade                       77159 non-null  float64
 10  earliest_cr_line            77159 non-null  object 
 11  issue_d                     77159 non-null  object 
 12  last_pymnt_d                77159 non-null  object 
 13  next_pymnt_d                332

In [43]:
remaining_categorical_features = transformed_loan_df.select_dtypes(exclude=[np.number])
print(len(remaining_categorical_features.columns))

7


In [54]:
categorical_columns_to_drop = categorical_features_of_interest
categorical_columns_to_drop.append('loan_status')
dropable_categorical_columns = list(set(transformed_loan_df.columns) - set(categorical_columns_to_drop))
dropable_categorical_columns = transformed_loan_df[dropable_categorical_columns].select_dtypes(exclude=[np.number]).columns
dropable_categorical_columns

Index(['initial_list_status', 'last_pymnt_d', 'next_pymnt_d', 'issue_d',
       'earliest_cr_line', 'pymnt_plan'],
      dtype='object')

In [55]:
encoded_loan_df = transformed_loan_df.drop(dropable_categorical_columns, axis=1)
encoded_loan_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77159 entries, 0 to 77158
Data columns (total 85 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          77159 non-null  int64  
 1   emp_title                   71297 non-null  float64
 2   application_type            77159 non-null  float64
 3   home_ownership              77159 non-null  float64
 4   loan_status                 77159 non-null  object 
 5   loan_amnt                   77159 non-null  int64  
 6   int_rate                    77159 non-null  float64
 7   term                        77159 non-null  float64
 8   installment                 77159 non-null  float64
 9   grade                       77159 non-null  float64
 10  annual_inc                  77159 non-null  float64
 11  verification_status         77159 non-null  float64
 12  purpose                     77159 non-null  float64
 13  hardship_flag               755

In [ ]:
print('45')